**Inspiring by [@ihower](https://ihower.tw/). Thanks!**

## OpenAI Dev Day 2023 Feature Demo

### Get OpenAI API Key from user imput

#### You can run this Jupyter Notebook with VSCode or Colab 

In [ ]:
%pip install pandas numpy shortuuid requests

In [61]:
## use getpass to read password, also save to the enviroment variable
import os, requests, json
from pprint import pp
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key:')

### Helper functions

In [62]:
def get_completion(messages, model='gpt-3.5-turbo', temperature=0, max_tokens=300):
    api_url = 'https://api.openai.com/v1/chat/completions'

    payload = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": messages,
    }

    headers = {
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
        "Content-Type": "application/json",
    }

    response = requests.post(api_url, headers=headers, data=json.dumps(payload), timeout=60)

    resp_text = json.loads(response.text)
    resp_status = response.status_code
    print(resp_text)
    print(resp_status)
    if resp_status == 200:
        return resp_text["choices"][0]["message"]["content"]
    else:
        return resp_text["error"]['message']

### Demo 1: GPT-4 Vision Experiment

#### Get random image from Lorem Picsum, preview

In [63]:
## preview the image_url
from IPython.display import Image
request_image_url = 'https://picsum.photos/800/1280'
response = requests.get(request_image_url, allow_redirects=True)
image_url = response.url

Image(url=image_url, width=200, height=300)

#### Generate text description

In [64]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "請以100字內描述這張照片"
            },
            {
                "type": "image_url",
                "image_url": image_url
            }
        ],
    }
]

completion = get_completion(messages, model='gpt-4-vision-preview')
pp(completion)

{'id': 'chatcmpl-8J1k1Wjg8p5XHUyladWfg1uwuGCCi', 'object': 'chat.completion', 'created': 1699545617, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 1128, 'completion_tokens': 108, 'total_tokens': 1236}, 'choices': [{'message': {'role': 'assistant', 'content': '這是一張黑白照片，展示了雙手在彈奏鋼琴。手指觸及黑白琴鍵，顯示出經驗的痕跡。照片背景模糊地顯示鋼琴品牌“Huberger & Co”。整體氛圍藝術而深沉。'}, 'finish_details': {'type': 'stop', 'stop': '<|fim_suffix|>'}, 'index': 0}]}
200
'這是一張黑白照片，展示了雙手在彈奏鋼琴。手指觸及黑白琴鍵，顯示出經驗的痕跡。照片背景模糊地顯示鋼琴品牌“Huberger & Co”。整體氛圍藝術而深沉。'


### Demo 2: Assistant API Demo

#### Step0: Upload a PDF file with OpenAI File API

In [65]:
# OpenAI assistant API with retrieval

## pdf from arXiv: https://arxiv.org/pdf/2305.06983.pdf

doc_source = 'https://arxiv.org/pdf/2305.06983.pdf'

filename = doc_source.split('/')[-1]

## check if the file exists
import os.path
if not os.path.isfile(filename):
    # wget with specific user agent
    !wget -U "Mozilla/5.0" -O $filename $doc_source
    print('downloaded file:', filename)    
    

files = [
    ('file',
      (
          filename,
          open(filename, 'rb'),
          "application/octet-stream"
      )
    )
]

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
}

data = {
    "purpose": "assistants",    
}

## List the files
# response = requests.get('https://api.openai.com/v1/files', headers=headers)
# print(json.loads(response.text))

### Upload the file to OpenAI

response = requests.post('https://api.openai.com/v1/files', headers=headers, data=data, files=files)

# print(json.loads(response.text))

## Get file id and save to the environment variable
file_id = json.loads(response.text)['id']
file_name = json.loads(response.text)['filename']
os.environ['OPENAI_FILE_ID'] = file_id

print(f"file_id: {file_id}, file_name: {file_name}")

file_id: file-t0mUO7iL7yRmeGKUjSs7CbPZ, file_name: 2305.06983.pdf


#### (Optional): List all uploaded files

In [66]:
## List the files
response = requests.get('https://api.openai.com/v1/files', headers=headers)
pp(json.loads(response.text))

{'object': 'list',
 'has_more': False,
 'data': [{'object': 'file',
           'id': 'file-t0mUO7iL7yRmeGKUjSs7CbPZ',
           'purpose': 'assistants',
           'filename': '2305.06983.pdf',
           'bytes': 850866,
           'created_at': 1699545644,
           'status': 'processed',
           'status_details': None},
          {'object': 'file',
           'id': 'file-DetKu8e8ZJWEu6Iweg988Oz8',
           'purpose': 'assistants',
           'filename': '停車場資訊.pdf',
           'bytes': 182055,
           'created_at': 1699444518,
           'status': 'processed',
           'status_details': None},
          {'object': 'file',
           'id': 'file-ffoOkEmb5gj3djWxvMJP8xYJ',
           'purpose': 'fine-tune-results',
           'filename': 'compiled_results.csv',
           'bytes': 215048,
           'created_at': 1675331528,
           'status': 'processed',
           'status_details': None},
          {'object': 'file',
           'id': 'file-XJ67TkoFSKM6QgwzsEr03YOl',
 

#### Step1: Create the assistant with retrieval
##### The indexing process will be started once the assistant is created.

In [67]:
import shortuuid

## Create the assistant with retrieval

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    "Content-Type": "application/json",
    "OpenAI-Beta": "assistants=v1",
}

### assistant_name should be unique with short UUID format
assistant_name = f'assistant-' + shortuuid.uuid()[0:8]

# assistant_name = 'jimmyliao-assistant-1'
instructions = """
    You are a customer support chatbot. Use your knowledge base to best respond to customer queries.
"""

payload = {
    "model": "gpt-4-1106-preview", 
    "name": assistant_name, 
    "tools": [{"type": "retrieval"}],
    "instructions": instructions,
    "file_ids": [file_id],
}

response = requests.post('https://api.openai.com/v1/assistants', headers=headers, data=json.dumps(payload))

assistant_id = json.loads(response.text)['id']
pp(json.loads(response.text))

{'id': 'asst_9OgwiVEppI8XRlEINALudBJ7',
 'object': 'assistant',
 'created_at': 1699545656,
 'name': 'assistant-h4AuruGg',
 'description': None,
 'model': 'gpt-4-1106-preview',
 'instructions': '\n'
                 '    You are a customer support chatbot. Use your knowledge '
                 'base to best respond to customer queries.\n',
 'tools': [{'type': 'retrieval'}],
 'file_ids': ['file-t0mUO7iL7yRmeGKUjSs7CbPZ'],
 'metadata': {}}


#### Step2: Create Thread
#### Step3: Add the user message to the thread

In [69]:
## Create thread 

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    "Content-Type": "application/json",
    "OpenAI-Beta": "assistants=v1",
}

### Add the user message to the thread
payload = { "messages": [ { "role": "user", "content": "write 100-word summary for this article."} ] }

response = requests.post('https://api.openai.com/v1/threads', headers=headers, data=json.dumps(payload))

thread_id = json.loads(response.text)['id']

pp(json.loads(response.text))

{'id': 'thread_0LJ5hfJySHh8FHF7UK1b4kgv',
 'object': 'thread',
 'created_at': 1699545669,
 'metadata': {}}


#### Step4: Run the assistant

In [70]:
## Execute (assistant + thread).run, and get the response

payload = {
    "assistant_id": assistant_id,
    # "thread_id": thread_id,
}

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    "Content-Type": "application/json",
    "OpenAI-Beta": "assistants=v1",
}

response = requests.post(f"https://api.openai.com/v1/threads/{thread_id}/runs", headers = headers, data = json.dumps(payload) )
obj = json.loads(response.text)

run_id = obj['id']
pp(obj)

{'id': 'run_xyJA4T24pY2Uub93ZNemRPfP',
 'object': 'thread.run',
 'created_at': 1699545674,
 'assistant_id': 'asst_9OgwiVEppI8XRlEINALudBJ7',
 'thread_id': 'thread_0LJ5hfJySHh8FHF7UK1b4kgv',
 'status': 'queued',
 'started_at': None,
 'expires_at': 1699546274,
 'cancelled_at': None,
 'failed_at': None,
 'completed_at': None,
 'last_error': None,
 'model': 'gpt-4-1106-preview',
 'instructions': '\n'
                 '    You are a customer support chatbot. Use your knowledge '
                 'base to best respond to customer queries.\n',
 'tools': [{'type': 'retrieval'}],
 'file_ids': ['file-t0mUO7iL7yRmeGKUjSs7CbPZ'],
 'metadata': {}}


#### (Optional): Monitor the Thread run status

In [71]:
## Monitor the run status

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    "Content-Type": "application/json",
    "OpenAI-Beta": "assistants=v1",
}

response = requests.get(f"https://api.openai.com/v1/threads/{thread_id}/runs/{run_id}", headers = headers)

obj = json.loads(response.text)
pp(obj)

{'id': 'run_xyJA4T24pY2Uub93ZNemRPfP',
 'object': 'thread.run',
 'created_at': 1699545674,
 'assistant_id': 'asst_9OgwiVEppI8XRlEINALudBJ7',
 'thread_id': 'thread_0LJ5hfJySHh8FHF7UK1b4kgv',
 'status': 'completed',
 'started_at': 1699545674,
 'expires_at': None,
 'cancelled_at': None,
 'failed_at': None,
 'completed_at': 1699545683,
 'last_error': None,
 'model': 'gpt-4-1106-preview',
 'instructions': '\n'
                 '    You are a customer support chatbot. Use your knowledge '
                 'base to best respond to customer queries.\n',
 'tools': [{'type': 'retrieval'}],
 'file_ids': ['file-t0mUO7iL7yRmeGKUjSs7CbPZ'],
 'metadata': {}}


#### Step5: Get the Assistant response from the thread

In [72]:
## Get the assistant response from thread

headers = { "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}", "Content-Type": "application/json", "OpenAI-Beta": "assistants=v1" }
response = requests.get(f"https://api.openai.com/v1/threads/{thread_id}/messages", headers = headers )
obj = json.loads(response.text)

pp(obj)


{'object': 'list',
 'data': [{'id': 'msg_Opx5NyEJiLxFLqzybgQAMOCM',
           'object': 'thread.message',
           'created_at': 1699545676,
           'thread_id': 'thread_0LJ5hfJySHh8FHF7UK1b4kgv',
           'role': 'assistant',
           'content': [{'type': 'text',
                        'text': {'value': 'The article introduces '
                                          'Forward-Looking Active REtrieval '
                                          'augmented generation (FLARE), a '
                                          'method enhancing large language '
                                          'models (LMs) to reduce '
                                          'hallucinations and factually '
                                          'incorrect outputs by actively '
                                          'retrieving information during text '
                                          'generation. Unlike traditional '
                                          'retrieval

#### (Optional) Add a new message to the thread, and run the Assistant Thread again

In [76]:
## Add a new message to the thread

payload = { "role": "user", "content": "有哪些作者？" }
headers = { "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}", "Content-Type": "application/json", "OpenAI-Beta": "assistants=v1" }
response = requests.post(f"https://api.openai.com/v1/threads/{thread_id}/messages", headers = headers, data = json.dumps(payload) )
obj = json.loads(response.text)

pp(obj)

## Execute (assistant + thread).run, and get the response

payload = {
    "assistant_id": assistant_id,
    # "thread_id": thread_id,
}

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    "Content-Type": "application/json",
    "OpenAI-Beta": "assistants=v1",
}

response = requests.post(f"https://api.openai.com/v1/threads/{thread_id}/runs", headers = headers, data = json.dumps(payload) )
obj = json.loads(response.text)

# run_id = obj['id']
pp(obj)

{'id': 'msg_AwmTYjYniBQtkNV84ISR0OVK',
 'object': 'thread.message',
 'created_at': 1699545850,
 'thread_id': 'thread_0LJ5hfJySHh8FHF7UK1b4kgv',
 'role': 'user',
 'content': [{'type': 'text', 'text': {'value': '有哪些作者？', 'annotations': []}}],
 'file_ids': [],
 'assistant_id': None,
 'run_id': None,
 'metadata': {}}
{'id': 'run_ze2nNTwTyMmQrIQ7px454kuP',
 'object': 'thread.run',
 'created_at': 1699545851,
 'assistant_id': 'asst_9OgwiVEppI8XRlEINALudBJ7',
 'thread_id': 'thread_0LJ5hfJySHh8FHF7UK1b4kgv',
 'status': 'queued',
 'started_at': None,
 'expires_at': 1699546451,
 'cancelled_at': None,
 'failed_at': None,
 'completed_at': None,
 'last_error': None,
 'model': 'gpt-4-1106-preview',
 'instructions': '\n'
                 '    You are a customer support chatbot. Use your knowledge '
                 'base to best respond to customer queries.\n',
 'tools': [{'type': 'retrieval'}],
 'file_ids': ['file-t0mUO7iL7yRmeGKUjSs7CbPZ'],
 'metadata': {}}


In [83]:
## Monitor the run status

headers = {
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    "Content-Type": "application/json",
    "OpenAI-Beta": "assistants=v1",
}

response = requests.get(f"https://api.openai.com/v1/threads/{thread_id}/runs/{run_id}", headers = headers)

obj = json.loads(response.text)
pp(obj)

{'id': 'run_ze2nNTwTyMmQrIQ7px454kuP',
 'object': 'thread.run',
 'created_at': 1699545851,
 'assistant_id': 'asst_9OgwiVEppI8XRlEINALudBJ7',
 'thread_id': 'thread_0LJ5hfJySHh8FHF7UK1b4kgv',
 'status': 'completed',
 'started_at': 1699545851,
 'expires_at': None,
 'cancelled_at': None,
 'failed_at': None,
 'completed_at': 1699545862,
 'last_error': None,
 'model': 'gpt-4-1106-preview',
 'instructions': '\n'
                 '    You are a customer support chatbot. Use your knowledge '
                 'base to best respond to customer queries.\n',
 'tools': [{'type': 'retrieval'}],
 'file_ids': ['file-t0mUO7iL7yRmeGKUjSs7CbPZ'],
 'metadata': {}}


#### Get the Assistant response from the thread

In [84]:
## Get the assistant response from thread with message id

headers = { "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}", "Content-Type": "application/json", "OpenAI-Beta": "assistants=v1" }

response = requests.get(f"https://api.openai.com/v1/threads/{thread_id}/messages", headers = headers )

obj = json.loads(response.text)

pp(obj)


{'object': 'list',
 'data': [{'id': 'msg_jvNyCQ4UaJRSZH6m7dtqQtHV',
           'object': 'thread.message',
           'created_at': 1699545861,
           'thread_id': 'thread_0LJ5hfJySHh8FHF7UK1b4kgv',
           'role': 'assistant',
           'content': [{'type': 'text',
                        'text': {'value': 'The authors of the article are '
                                          'Zhengbao Jiang, Frank F. Xu, Luyu '
                                          'Gao, Zhiqing Sun, Qian Liu, Jane '
                                          'Dwivedi-Yu, Yiming Yang, Jamie '
                                          'Callan, and Graham '
                                          'Neubig【20†source】.',
                                 'annotations': [{'type': 'file_citation',
                                                  'text': '【20†source】',
                                                  'start_index': 154,
                                                  'end_index': 165,
  